# Рекуррентные нейронные сети (RNN)

##  Особенности

Если стандартные методы машинного обучения работают с так называемыми структурированными типами данных, с объектами, у которых фиксировано признаковое описание, которые подаются на вход моделям, то рекуррентные нейронные сети используются с неструктурированными данными. Например, в распознавании речи, генерации музыки, машинном переводе. Данные в таких задачах представляют собой текст без предобработки, звуковой ряд и другие данные, не имеющие признаковых описаний. Давайте обсудим, каков принцип работы нейронной сети, и как именно таким нейронным сетям удается работать с неструктурированными данными.

Мы имеем возможность сохранять информацию, сформированную при обработке одного слова, и использовать ее, когда мы анализируем дальнейшие слова. Видно, что при рассмотрении каждого слова происходят одни и те же процедуры. Нейронная сеть получает очередное слово, учитывает предыдущую активацию, формирует ответ. Это действие, происходящее в рамках одной ячейки нейронной сети. Так как вся сеть представляет собой объединение однотипных ячеек, то нейронная сеть называется рекуррентной. Рекуррентные нейронные сети не ограничиваются случаем, когда мы каждому входу нейронной сети должны сопоставить некоторый ответ, после чего анализировать следующий вход.

## Задачи

**Анализ временных рядов**
- Статистические данные
- Аннотирование изображений и видео (Image/Video captioning)
- Машинный перевод
- Распознавание текста
- Распознавание речи

**Генеративные модели**
- Генерация текста/речи (чат - боты)
- Генерация изображений

**Классификация**
- Изображения
- Блоки текста (Sentiment analysis)

Большинство этих задачь подрузамевают наличие контекста ...

<img src ="http://edunet.kea.su/repo/src/L08_RNN/img/498_FA2019_lecture12-011.png" width="700">

Существует множество типов рекуррентных нейронных сетей. Простейший случай рекуррентной сети — «one to one», когда у нас есть всего один вход и один выход нейронной сети. Более сложной является реализация «one to many», когда у нас есть всего один вход и нам необходимо сформировать несколько выходов. Такой тип нейронной сети актуален, когда мы говорим о генерации музыки или текстов. Мы задаем начальное слово или начальный звук, а дальше модель начинает самостоятельно генерировать выходы, в качестве входа к очередной ячейке рассматривая выход с прошлой ячейки нейронной сети. Если мы рассматриваем задачу классификации, то актуальна схема «many to one». Мы должны проанализировать все входы нейронной сети и только в конце определиться с классом. Схему «many to many», когда количество выходов равно количеству входов нейронной сети, мы рассмотрели на примере с определением части речи. Такой вид используется также в задачах NER, которые мы обсудим в следующем видео. Ну и последней разновидностью нейронных сетей является сеть вида «many to many», когда количество выходов нейронной сети не равно количеству входов. Это актуально, к примеру, в машинном переводе, когда одна и та же фраза может иметь разное количество слов в разных языках (т.е. это реализует схему энкодер-декодер). Энкодер получает данные различной длины — например, предложение на английском языке. С помощью скрытых состояний он формирует из исходных данных вектор, который затем передаётся в декодер. Последний, в свою очередь, генерирует из полученного вектора выходные данные — исходную фразу, переведённую на другой язык.

## Базовый RNN блок

Попробуем подробнее разобраться, что же происходит в загадочном зелёном прямоугольнике с надписью RNN. Внутри него мы вычисляем рекуррентное соотношение с помощью функции f, которая зависит от весов w. Чтобы найти новое состояние ht, мы берём предыдущее скрытое состояние $ h_{t-1} $, а также текущий ввод xt. Когда мы отправляем в модель следующие входные данные, полученное нами скрытое состояние $ h_t $ передаётся в эту же функцию, и весь процесс повторяется.

Чтобы генерировать вывод в каждый момент времени, в модель добавляются полносвязные слои, которые постоянно обрабатывают состояния $ h_t $ и выдают основанные на них прогнозы. При этом функция f и веса w остаются неизменными.

Самая простая реализация рекуррентной сети будет выглядеть следующим образом (Тангенс здесь используется для введения нелинейности в систему):

<img src ="http://edunet.kea.su/repo/src/L08_RNN/img/498_FA2019_lecture12-017.png" width="700">

Важное отличие от слоев с которыми мы уже сталкивались, сотоит в том что на выходе мы получаем два объекта Y и H.

**Y** - предсказание в текущий момент времени, например метка класса.

**H** - контекст в котором предсказание было сделанно. Он может использоваться для дальнейших предсказаний.

#### RNNCell

В Pytorch для вычисления h_t используется модуль [RNNCell](https://pytorch.org/docs/stable/generated/torch.nn.RNNCell.html)  

y_t в нем не вычисляется, предполагается что для его получения в модель должен быть добавлен дополнительный линейный слой.

In [ ]:
import torch

rnn_cell = torch.nn.RNNCell(input_size = 3, hidden_size = 2)
dummy_sequence = torch.randn((1,3)) # batch, input_size
h = rnn_cell(dummy_sequence) 
print("Out = h\n",h.shape,"\n",h) # hidden state 

Внутри происходит примерно следующее:

In [ ]:
import numpy as np
from torch import nn

# Simple RNNcell without a bias and batch support
class SimplifiedRNNCell(nn.Module): 
  def __init__(self, input_size, hidden_size):
    super().__init__()
    # hidden_size == number of neurons
    self.W_hx = np.random.randn(input_size, hidden_size) * 0.0001 # without bias
    self.W_hh = np.random.randn(hidden_size, hidden_size) * 0.0001 # without bias
    self.h0 = np.zeros((hidden_size))
  
  def forward(self,x,h = None): # Without a batch dimension
    if h is None:
      h = self.h0
    h = np.tanh(self.W_hx.T.dot(x)+self.W_hh.T.dot(h))
    return h 
  
simple_rnn_cell = SimplifiedRNNCell(input_size = 3, hidden_size = 2)
h = simple_rnn_cell(dummy_sequence[0]) # No batch 
print("Out = h\n",h.shape,"\n",h) 


<img src ="http://edunet.kea.su/repo/src/L08_RNN/img/498_FA2019_lecture12-023.png" width="700">

Однако в последовательности всегда несколько элементов. И надо применить алгоритм к каждому.


 Поэтому RNNCell напрямую не используется. Для него есть обертка: [RNN](https://pytorch.org/docs/stable/generated/torch.nn.RNN.html) который обеспечивает последовательный вызов RNNCell для всех элементов последовательности.

### RNN блок в Pytorch 

**Warning: batch dim is second!**

In [ ]:
from torch import nn
rnn = torch.nn.RNN(input_size = 3, hidden_size = 2) #batch_first = True
dummy_batched_seq = torch.randn((2,1,3)) #   seq_len, batch , input_size
out, h = rnn(dummy_batched_seq) 
print("Out = \n",out.shape,"\n",out) # hidden state for each element of sequence
print("h = \n",h.shape,"\n",h) # hidden state for last element of sequence

Внутри происходит примено следующее

In [ ]:
# Simple RNN without batching
import numpy as np
from torch import nn

class SimplifiedRNNLayer(nn.Module): 
  def __init__(self, input_size, hidden_size):
    super().__init__()
    self.rnn_cell = SimplifiedRNNCell(input_size, hidden_size)

  # Without a batch dimension x have sahpe seq_len * input_size
  def forward(self,x, h = None):
    all_h = []
    for i in range(x.shape[0]):
      h = self.rnn_cell(x[i],h)
      all_h.append(h) 
    return  np.stack(all_h), h

simple_rnn = SimplifiedRNNLayer(input_size = 4, hidden_size = 2)
sequence = np.array([[0,1,2,0], [3,4,5,0]]) # batch of one sequence of two elements 

out, h = simple_rnn(sequence)
print("Out \n",out.shape,out) 
print("h \n", h.shape, h)

Давайте разберемся.

К данным добаляется еще одно измерение размер последовательности. Таким образом batch из 6 последовательностей по 5 элементов в каждой будет выглядеть так:
<img src ="http://edunet.kea.su/repo/src/L08_RNN/img/gan/rnn_batch.jpeg" width="600">

P.S. Размер самого элемента == 3

Внутри RNN модуля элеменым последовательности обрабатываются последовательно:

<img src ="http://edunet.kea.su/repo/src/L08_RNN/img/gan/rnn_unrolled.jpeg" width="1000">

Веса при этом используются одни и те же.

In [ ]:
import torch

dummy_input = torch.randn((2,1,3)) #  seq_len, batch, , input_size

print("RNNCell")
rnn_cell = torch.nn.RNNCell(3,2)
for t, p in rnn_cell.named_parameters():
  print(t, p.shape) 

cell_out = rnn_cell(dummy_input[0,:,:]) # take first element from sequence 
print("Out = h",cell_out) # one hidden state

print("RNN")
rnn = torch.nn.RNN(3,2)
for t, p in rnn_cell.named_parameters():
  print(t, p.shape) 

out, h = rnn(dummy_input)

print("Out", out) # h for all sequence element 
print("h", h) # h for last element 

Кроме этого RNN блок имеет еще ряд настроек:

<img src ="http://edunet.kea.su/repo/src/L08_RNN/img/l08_add1.png" width="700">
<img src ="http://edunet.kea.su/repo/src/L08_RNN/img/l08_add2.png" width="700">
<img src ="http://edunet.kea.su/repo/src/L08_RNN/img/l08_add3.png" width="700">

### Слои (Stacked RNNs)

https://discuss.pytorch.org/t/what-is-num-layers-in-rnn-module/9843/2


<img src ="http://edunet.kea.su/repo/src/L08_RNN/img/gan/layers.png" width="700">


In [ ]:
import torch

dummy_input = torch.randn((2,1,3)) #  seq_len,  batch, input_size
rnn = torch.nn.RNN(3,2,num_layers=3)

# Weights matrix sizes not changed!
for t, p in rnn_cell.named_parameters():
  print(t, p.shape) 

out, h = rnn(dummy_input)

print("Out", out) # Hidden states for all elements from top layer
print("h", h) # Hidden states for last element for all layers

https://medium.com/dair-ai/building-rnns-is-fun-with-pytorch-and-google-colab-3903ea9a3a79j

### Bidirectional

Последовательность можно пропустить через сетьдва раза в прямом и обратном направлении.

https://medium.com/analytics-vidhya/understanding-rnn-implementation-in-pytorch-eefdfdb4afdb

<img src ="http://edunet.kea.su/repo/src/L08_RNN/img/gan/bidirectional.png" width="700">

In [ ]:
import torch

dummy_input = torch.randn((2,1,3)) #   seq_len, batch, input_size
rnn = torch.nn.RNN(3,2,bidirectional=True,num_layers=1)

for t, p in rnn_cell.named_parameters():
  print(t, p.shape) 

out, h = rnn(dummy_input)

print("Out", out) # Concatenated Hidden states from both layers
print("h", h) # Hidden states last element from  both : 2*num_layers*hidden_state

### Добавление выходных весов (y_t)

Давайте добавим выходные веса. Для этого придется программировать.Воспользуемся параметром batch_first = True что бы batch измерение оказалось на привычном нам месте.

In [ ]:
import torch
# Let's add output weights

class RNN_for_many_to_one(torch.nn.Module):
    def __init__(self, input_size, hidden, output_size):
        super().__init__()
        self.rnn = torch.nn.RNN(input_size, hidden, batch_first = True)
        self.fc1 = torch.nn.Linear(hidden, output_size)
    
    def forward(self, x):
        x, hidden = self.rnn(x)
        print(x.shape) # h for each element
        print(hidden.shape) 
        # we need only last output
        #return self.fc1(x[-1])
        return self.fc1(hidden)
    
model2 = RNN_for_many_to_one(28, 128, 10) # input_size, hidden_dim, classes
dummy_input = torch.randn((8,28,28)) #  seq_len , batch, element_size
res = model2(dummy_input)
# 30 - batch, 50 - output_size
print(res.shape)

По умолчанию batch_first =  False и batch измерение становится вторым!

## Many to one

## Пример обработки временного ряда


Что общего у прогнозирования потребления электроэнергии домохозяйствами, оценки трафика на дорогах в определенные периоды, прогнозировании паводков и прогнозировании цены, по которой акции будут торговаться на фондовой бирже?

Все они подпадают под понятие данных временных рядов! Вы не можете точно предсказать любой из этих результатов без компонента «время». И по мере того, как в мире вокруг нас генерируется все больше и больше данных, прогнозирование временных рядов становится все более важной областью применения методов ML и DL.



### Загрузка данных

Air Passengers per month. 
https://www.kaggle.com/rakannimer/air-passengers



In [ ]:
# Dataloading
import pandas as pd

!if test -f ./airline-passengers.csv; then echo "Already downloaded"; else wget https://raw.githubusercontent.com/jbrownlee/Datasets/master/airline-passengers.csv; fi

dataset = pd.read_csv('airline-passengers.csv')
dataset.head()

In [ ]:
import matplotlib.pyplot as plt
training_data = dataset.iloc[:,1:2].values # перевели dataframe в numpy.array
# plotting
plt.figure(figsize=(12, 4))
plt.plot(training_data, label = 'Airline Passangers Data')
plt.grid()
plt.show()

### Предобработка данных

In [ ]:
# Min - Max normalization
td_min = training_data.min()
td_max = training_data.max()
training_data -= td_min
training_data = training_data / td_max
print(training_data[:5])

### Подготовка данных

Поскольку мы хотим научиться предсказывать следующие значение на основе предидущих нам нужно подготовить данные соответствующим образом.

Разобьем весь массив данных на фрагменты вида

x -> y

где х - это подпоследовательность, например строки с 1-й по 8ю, а y - это значение из 9-й строки, то самое которое мы хотим предсказать




In [ ]:
import numpy as np
# функция создания "ансамблей" данных
def sliding_windows(data, seq_length):
    x = []
    y = []

    for i in range(len(data)-seq_length-1):
        _x = data[i:(i+seq_length)] # seq_len * elements
        _y = data[i+seq_length] # one element
        x.append(_x)
        y.append(_y)

    return np.array(x),np.array(y)
    
# установка длины ансамбля. от нее практически всегда зависит точность предикта и быстродействие
seq_length = 8 # сравните 2 и 32
x, y = sliding_windows(training_data, seq_length)
x[0], y[0]

Благодаря такому подходу мы можем работать с RNN моделью так же как работали со сверточными моделями. Подавая на вход такую подпоследовательность + результат.

### Разобьем на train и test

In [ ]:
train_size = int(len(y) * 0.8)
test_size = len(y) - train_size

dataX = torch.Tensor(np.array(x))
dataY = torch.Tensor(np.array(y))

trainX = torch.Tensor(np.array(x[0:train_size]))
trainY = torch.Tensor(np.array(y[0:train_size]))

testX = torch.Tensor(np.array(x[train_size:len(x)]))
testY = torch.Tensor(np.array(y[train_size:len(y)]))

print(trainX.shape, trainY.shape, testX.shape, testY.shape)

### Model


In [ ]:
import torch
import torch.nn as nn

class AirTrafficPredictor(nn.Module):

    def __init__(self, input_size, hidden_size):
        # hidden_size == number of neurons 
        super().__init__()
        self.rnn = nn.RNN(input_size=input_size, hidden_size=hidden_size, batch_first = True)
        self.fc = nn.Linear(hidden_size, 1) # Predict only one value

    def forward(self, x):
        #print("x: ",x.shape) # 108 x 8 x 1 : [batch_size, seq_len, input_size] 
        out, h = self.rnn(x) 
        #print("out: ", out.shape) # 108 x 8 x 4 : [batch_size, seq_len, hidden_size] Useless!
        #print("h : ", h.shape) # 1 x 108 x 4 [ num_layers, batch_size, hidden_size]
        y = self.fc(h)
        #print("y",y.shape) # 1 x 108 x 1
        return y, h

### Обучение

Благодаря подготовке данных процесс обучения не будет отличаться от того что мы использовали на прошедших занятиях.

В силу того что датасет маленький, все данные поместились в один batch, итерирования по batch-ам в явном виде здесь не происходит.

In [ ]:
def time_series_train(model):
  num_epochs = 2000
  learning_rate = 0.01
  

  criterion = torch.nn.MSELoss() # mean-squared error for regression
  optimizer = torch.optim.Adam(rnn.parameters(), lr=learning_rate)

  # Train the model
  for epoch in range(num_epochs):
      outputs, h = model(trainX) # we don't use h there, but we can!
      optimizer.zero_grad()
      
      # obtain the loss function
      loss = criterion(outputs, trainY)
      loss.backward()
      
      optimizer.step()
      if epoch % 100 == 0:
          print("Epoch: %d, loss: %1.5f" % (epoch, loss.item()))

input_size = 1
hidden_size = 4 
rnn = AirTrafficPredictor(input_size, hidden_size)
time_series_train(rnn)

### Testing

In [ ]:
def time_series_plot(train_predict):
  data_predict = train_predict.data.numpy()
  dataY_plot = dataY.data.numpy()

  # Denormalize
  data_predict = data_predict[0] *td_max + td_min
  dataY_plot = dataY_plot *td_max + td_min 
  #print(data_predict[:15])

  # Ploitting
  plt.figure(figsize=(12, 4))
  plt.axvline(x=train_size, c='r', linestyle='--')

  plt.plot(dataY_plot)
  plt.plot(data_predict)
  plt.suptitle('Time-Series Prediction')
  plt.show()

rnn.eval()
train_predict, h = rnn(dataX)
time_series_plot(train_predict)




[Time Series Prediction with LSTM Using PyTorchTime Series Prediction with LSTM Using PyTorch](https://colab.research.google.com/github/dlmacedo/starter-academic/blob/master/content/courses/deeplearning/notebooks/pytorch/Time_Series_Prediction_with_LSTM_Using_PyTorch.ipynb#scrollTo=NabsV8O5BBd5https://colab.research.google.com/github/dlmacedo/starter-academic/blob/master/content/courses/deeplearning/notebooks/pytorch/Time_Series_Prediction_with_LSTM_Using_PyTorch.ipynb#scrollTo=NabsV8O5BBd5)



## Типы RNN архитектур

<img src ="http://edunet.kea.su/repo/src/L08_RNN/img/498_FA2019_lecture12-026.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L08_RNN/img/498_FA2019_lecture12-028.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L08_RNN/img/498_FA2019_lecture12-030.png" width="700">

## Посимвольная генерация текстов

https://github.com/gabrielloye/RNN-walkthrough/blob/master/main.ipynb

Одним из основных направлений использования рекуррентных сетей является работа с текстами:
- генерация (Language modeling)
и 
- перевод (Machine Translation)

Давайте посмотрим как решаются такого рода задачи.

Начнем с относительно простой - посимвольной генерации текста.

Постановка задачи:

предсказать следующий символ в последовательности.

- исходный текст:
'hey how are you'

- искаженный текст:

'hey how are yo'

- Верное предсказание:
'u'


Теоретически эту технику можно использовать для генерации подсказок при наборе текстов, исправления ошибок или восстановления частично утраченного текста.





<img src ="http://edunet.kea.su/repo/src/L08_RNN/img/498_FA2019_lecture12-037.gif" width="400">

#### Подготовка данных

1. Зафиксировать словарь
2. Разбить данные
3. Кодирование символов 

In [ ]:
text = ['hey how are you','good i am fine','have a nice day']

# Join all the sentences together and extract the unique characters from the combined sentences
chars = set(''.join(text))

# Creating a dictionary that maps integers to the characters
int2char = dict(enumerate(chars))

# Creating another dictionary that maps characters to integers
char2int = {char: ind for ind, char in int2char.items()}

print(char2int)

Вместо ascii символа, каждой букве мы сопоставили номер.

### Выравнивание данных (Padding)

RNN допускают работу с данными переменной длины. Но что бы поместить предложения в batch надо их выровнять.


Обычно размер батча делают равным самому длинному предложению, а остальные просто дополняют пробелами до этого размера.

In [ ]:
maxlen = len(max(text, key=len))
print("The longest string has {} characters".format(maxlen))

# A simple loop that loops through the list of sentences and adds a ' ' whitespace until the length of the sentence matches
# the length of the longest sentence
for i in range(len(text)):
    while len(text[i])<maxlen:
        text[i] += ' '

print(text)



### Разбиение данных

Можно как в случае с временым рядом предсказывать отдельный символ (many_to_one).

Но мы используем many_to_many что бы увидеть как иначе можно обработать данные.

В качестве входа будем использовать предложение без последнего символа:

**'hey how are yo'**

, а в качестве результата - предложение в котором он сгенерирован.

**'ey how are you'**


In [ ]:
# Creating lists that will hold our input and target sequences
input_seq = []
target_seq = []

for i in range(len(text)):
    # Remove last character for input sequence
    input_seq.append(text[i][:-1])
    
    # Remove firsts character for target sequence
    target_seq.append(text[i][1:])
    print("Input Sequence: {}\nTarget Sequence: {}".format(input_seq[i], target_seq[i]))

Как видим выавнивание служит здесь плохую службу.

### Кодирование

Теперь символы надо перевести в числа. Для этого мы уже построили словарь.

P.S. Запускать блок только один раз.

In [ ]:
for i in range(len(text)):
    input_seq[i] = [char2int[character] for character in input_seq[i]]
    target_seq[i] = [char2int[character] for character in target_seq[i]]

print("Input",input_seq)
print("Target",input_seq)

#### One-hot-encoding(!)

Теперь из чисел надо сделать вектора. 


Почему бы не оставить числа?
В прошлом примере модель хорошо с ними работала.

В прошлом примере использовася MSE и на выходе было число.

Если бы мы определили отношение порядка над номерами букв, то что-то подобное можно было бы сделать.

Однако сейчас мы предсказываем класс буквы.
Поэтому на входе и на выходе должен быть вектор.

<img src ="http://edunet.kea.su/repo/src/L08_RNN/img/gan/softmax_1.png" width="200">








In [ ]:
dict_size = len(char2int)
seq_len = maxlen - 1
batch_size = len(text)

def one_hot_encode(sequence, dict_size, seq_len, batch_size):
    # Creating a multi-dimensional array of zeros with the desired output shape
    features = np.zeros((batch_size, seq_len, dict_size), dtype=np.float32)
    
    # Replacing the 0 at the relevant character index with a 1 to represent that character
    for i in range(batch_size):
        for u in range(seq_len):
            features[i, u, sequence[i][u]] = 1
    return features

input_seq = one_hot_encode(input_seq, dict_size, seq_len, batch_size)
print("Input shape: {} --> (Batch Size, Sequence Length, One-Hot Encoding Size)".format(input_seq.shape))
print(input_seq[0])

Каждый символ закодировали вектором.
Не слишьком экономно, зато удобно умножать на матрицу весов.

P.S. Запускать только один раз

In [ ]:
# Convert data to tensor
input_seq = torch.from_numpy(input_seq)
target_seq = torch.Tensor(target_seq)

### Модель

In [ ]:
class NextCharacterGenerator(nn.Module):
    def __init__(self, input_size, output_size, hidden_dim, n_layers):
        super().__init__()
        #self.n_layers = n_layers

        # RNN Layer
        self.rnn = nn.RNN(input_size, hidden_size = hidden_dim, batch_first=True)   
        # Fully connected layer
        self.fc = nn.Linear(hidden_dim, output_size)
    
    def forward(self, x):
        batch_size = x.size(0)
        #Initializing hidden state for first input using method defined below
        hidden_0 = torch.zeros(1, batch_size, self.rnn.hidden_size) # 1 correspond to number of layers

        # Passing in the input and hidden state into the model and obtaining outputs
        out, hidden = self.rnn(x, hidden_0)
        
        # Reshaping the outputs such that it can be fit into the fully connected layer
        # Only if n_layers > 1
        out = out.contiguous().view(-1, self.rnn.hidden_size)
        out = self.fc(out)
        
        return out, hidden

### Обучение

In [ ]:
# Instantiate the model with hyperparameters
model = NextCharacterGenerator(input_size=dict_size, output_size=dict_size, hidden_dim=12, n_layers=1)

# Define hyperparameters
n_epochs = 100

# Define Loss, Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

# Training Run
for epoch in range(1, n_epochs + 1):
    optimizer.zero_grad() # Clears existing gradients from previous epoch
    output, hidden = model(input_seq)
    loss = criterion(output, target_seq.view(-1).long())
    loss.backward() # Does backpropagation and calculates gradients
    optimizer.step() # Updates the weights accordingly
    
    if epoch%10 == 0:
        print('Epoch: {}/{}.............'.format(epoch, n_epochs), end=' ')
        print("Loss: {:.4f}".format(loss.item()))

Тест

In [ ]:
def predict(model, character):
    # One-hot encoding our input to fit into the model
    character = np.array([[char2int[c] for c in character]])
    character = one_hot_encode(character, dict_size, character.shape[1], 1)
    character = torch.from_numpy(character)
    
    out, hidden = model(character)
    print(out.shape)
    #print(out)
    prob = nn.functional.softmax(out[-1], dim=0).data
    # Taking the class with the highest probability score from the output
    char_ind = torch.max(prob, dim=0)[1].item()

    return int2char[char_ind], hidden

def sample(model, out_len, start='hey'):
    model.eval() # eval mode
    start = start.lower()
    # First off, run through the starting characters
    chars = [ch for ch in start]
    size = out_len - len(chars)
    # Now pass in the previous characters and get a new one
    for ii in range(size):
        char, h = predict(model, chars)
        chars.append(char)

    return ''.join(chars)

sample(model, 15, 'good')

<img src ="http://edunet.kea.su/repo/src/L08_RNN/img/gan/softmax_2.png" width="700">

Результаты которые удается получить при помощи моделей обученных на больших объемах данных.

<img src ="http://edunet.kea.su/repo/src/L08_RNN/img/498_FA2019_lecture12-050.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L08_RNN/img/498_FA2019_lecture12-054.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L08_RNN/img/498_FA2019_lecture12-055.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L08_RNN/img/498_FA2019_lecture12-058.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L08_RNN/img/498_FA2019_lecture12-059.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L08_RNN/img/498_FA2019_lecture12-061.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L08_RNN/img/498_FA2019_lecture12-063.png" width="700">

### Explaination

<img src ="http://edunet.kea.su/repo/src/L08_RNN/img/498_FA2019_lecture12-064.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L08_RNN/img/498_FA2019_lecture12-065.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L08_RNN/img/498_FA2019_lecture12-066.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L08_RNN/img/498_FA2019_lecture12-067.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L08_RNN/img/498_FA2019_lecture12-068.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L08_RNN/img/498_FA2019_lecture12-069.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L08_RNN/img/498_FA2019_lecture12-068.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L08_RNN/img/498_FA2019_lecture12-069.png" width="700">

##Обратное распространение

<img src ="http://edunet.kea.su/repo/src/L08_RNN/img/498_FA2019_lecture12-045.png" width="700">

Теоритетески можно было бы сразу пропустить все данные через сеть и затем вычислить градиент.

Но:
 - Большие последовательности не поместятся в памяти
 - Возникнут проблеммы исчезновения/взрыва градиента, так как цепочка будет очень длинной




### Пакеты



<img src ="http://edunet.kea.su/repo/src/L08_RNN/img/498_FA2019_lecture12-046.png" width="700">

Поэтому снова возникает потребность в обработке по батчам.

<img src ="http://edunet.kea.su/repo/src/L08_RNN/img/498_FA2019_lecture12-047.png" width="700">

Скрытые состояния при этом сохраняются.

<img src ="http://edunet.kea.su/repo/src/L08_RNN/img/498_FA2019_lecture12-083.png" width="700">

Градиент в рекуррентных сетях затухает так же быстро. И при большой длине последовательности это становится критичным.

<img src ="http://edunet.kea.su/repo/src/L08_RNN/img/498_FA2019_lecture12-086.png" width="700">

Исчезающий/взрывающийся градиент (Vanishing/exploding gradient) - явления исчезающего и взрывающегося градиента часто встречаются в контексте RNN. Причина, по которой они происходят, заключается в том, что трудно уловить долгосрочные зависимости из-за мультипликативного градиента, который может экспоненциально уменьшаться/увеличиваться по отношению к числу слоев.

Градиентное отсечение (Gradient clipping) - метод, используемый для решения проблемы взрывающегося градиента, иногда возникающей при выполнении обратного распространения. Ограничивая максимальное значение градиента, это явление контролируется на практике.

<img src ="http://edunet.kea.su/repo/src/L08_RNN/img/gradient-clipping.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L08_RNN/img/gradient-vanishing-exploding.png" width="700">

## LSTM



<img src ="http://edunet.kea.su/repo/src/L08_RNN/img/498_FA2019_lecture12-090.png" width="700">

Если в Vanilla RNN был только один путь, то в LSTM есть highway для сокращения информации  

<img src ="http://edunet.kea.su/repo/src/L08_RNN/img/gan/lstm.png" width="700">

i = Input
f = Forget
o = Output
g = Gate



1. Конкатенируем x и h_t-1
2. Умножаем на веса
3. Результат делим на 4 части (shape = hidden_size) к каждой применяем свою функцию активации
4. Далее комбинируем их с входами и выходами


<img src ="http://edunet.kea.su/repo/src/L08_RNN/img/498_FA2019_lecture12-094.png" width="700">

Но нам все равно не нужно распространять через веса (потенциально это может вызвать проблемы)

<img src ="http://edunet.kea.su/repo/src/L08_RNN/img/498_FA2019_lecture12-095.png" width="700">

### LSTMCell

https://pytorch.org/docs/stable/generated/torch.nn.LSTMCell.html

Интерфейс отличается от RNNCell количеством входов и выходов


In [ ]:
import torch

lstm_cell = torch.nn.LSTMCell(input_size = 3, hidden_size = 4)
dummy_input = torch.randn(1,3) # batch, input_size
h_0 = torch.randn(1,4)
c_0 = torch.randn(1,4)
h, c = lstm_cell(dummy_input, (h_0,c_0)) # second arg is tuple
print("h",h.shape,h) # batch, hidden_size
print("c",c.shape,c) # batch, hidden_size



### LSTM in Pytorch

Отличие состоит в том что возвращается кроме h возвращается еще и c. Но можно использовать только output. 

In [ ]:
import torch
lstm = nn.LSTM(input_size = 3, hidden_size = 3)
dummy_input = torch.randn(2,1,3) # seq_len, batch, input_size
out, (h, c) = lstm(dummy_input) # h and c returned in tuple
print("out",out.shape,out) # seq_len, batch, hidden_size : h for each element
print("h",h.shape,h) # batch, hidden_size
print("c",c.shape,c) # batch, hidden_size

### Пример использования 

Что бы убедиться в работоспособности конструкции заменим RNN блок на LSTM в задаче предсказания временного ряда.

In [ ]:
# Define new LSTM based model
import torch
import torch.nn as nn

class LSTMAirTrafficPredictor(nn.Module):

    def __init__(self, input_size, hidden_size):
        # hidden_size == number of neurons 
        super().__init__()
        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size, batch_first = True)
        self.fc = nn.Linear(hidden_size, 1) # Predict only one value

    def forward(self, x):
        out, (h,c) = self.lstm(x) 
        y = self.fc(h)
        return y

lstm =  LSTMAirTrafficPredictor(input_size =1 , hidden_size =4 )
di = torch.randn((108,8,1))
out = lstm(di)
print(out.shape)


Train

In [ ]:
lstm.train()

num_epochs = 2000
learning_rate = 0.01

criterion = torch.nn.MSELoss() # mean-squared error for regression
optimizer = torch.optim.Adam(lstm.parameters(), lr=learning_rate)

 # Train the model
for epoch in range(num_epochs):
    outputs = lstm(trainX) 
    optimizer.zero_grad()
    #print(outputs.shape)
    loss = criterion(outputs, trainY.unsqueeze(0))
    loss.backward()
    
    optimizer.step()
    if epoch % 100 == 0:
        print("Epoch: %d, loss: %1.5f" % (epoch, loss.item()))


In [ ]:
lstm.eval()
train_predict = lstm(dataX)
time_series_plot(train_predict)


## GRU (Gated reccurent unit)

<img src ="http://edunet.kea.su/repo/src/L08_RNN/img/498_FA2019_lecture12-101.png" width="700">

Были предприняты попытки с помощью эволюционного поиска найти более оптимальный юнит для RNN. 

*Поиск шел в пространстве формул для обновления состояний*

Тем не менее, никакого значительного улучшения качва достигнуто не было относительно LSTM, именно поэтому LSTM является оптимальой архитектурой.

## Architecture search

<img src ="http://edunet.kea.su/repo/src/L08_RNN/img/498_FA2019_lecture12-102.png" width="700">

## Sequence-to-Sequence with RNNs

* Сейчас мы пытаемся решить задачу sequence to sequence
* Орабатывая входную последовательность, мы хотим обобщить всю информацию, которая в ней содержится в некий вектор С
* Далее мы передаем этот вектор во вторую RNN, которая является декодером


Мы используем вектор С для передачи информации между энкодером и декодером, также мы предполагаем, что вектор С обощает всю информацию, которая небходима декодеру для генерации выходных последовательности.

Однако использование единственного вектора для предоставления информации может работать только для последовательностей малой длины (неразумно предполагать, что информация с длинной последовательностью можно сжать в один вектор) 

Именно поэтому нам нужен некий механизм, который будет обходить "бутылочное горлышко" и не сжимать всю информацию в один вектор С. 

<img src ="http://edunet.kea.su/repo/src/L08_RNN/img/498_FA2019_lecture13-011.png" width="700">

## Attention

### Sequence-to-Sequence with RNNs and Attention mechanism

Давайте позволим нашему декодеру, используя context vector, фокусироваться на определенных выходах энкодера. 

Для этого будем вычислять context vector и вводим некую фцию $F_{att}$ , котрая будет вычислять alignment score (функцию похожести) между внутренними состояниями декодера и энкодера.

После этого, вычисленные alignment scores мы пропускаем через softmax для нормализации. Таким образом мы получаем веса, которые говорят о том, насколько сильно мы хотим сфокусироваться на соответствующим им выходам энкодера.

Затем мы просто берем взевенную сумму внутренних состояний энкодера и передаем ее на вход декодеру. Данный алгоритм выполняем на каждом шаге декодера.

Функция $F_{att}$ может являться небольшой нейросетью или неким линейным преобразованием. Однако это не так важно, идея заключается в том, что мы не говорим явно на чем функция должна сосредотачиваться в конкретной задаче. Применив к ней back propagation, ее веса сами настроятся на необходимые точки фокусировки.

**То есть функция сама разберется на чем фокусироваться**



<img src ="http://edunet.kea.su/repo/src/L08_RNN/img/498_FA2019_lecture13-017.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L08_RNN/img/498_FA2019_lecture13-020.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L08_RNN/img/498_FA2019_lecture13-021.png" width="700">

Давайте посмотрим как фокусировка работает на примере перевода с английского на французский 

Как мы знаем, на каждом шаге генерируется набор весов, которые отвечают за фокусировку на том или ином месте входной последовательности. Как мы видим, английское предложение имеет иной порядок слов относительно французского. Например, в английском варианте словосочетание **European Economic Area**, в то время как во французском **zone économique européenne**. 

В английском прилагательные идут перед существительным, в то время как во французском языке наоборот. 

Таким образом, благодаря гибкости модели, мы можем обрабатывать и учитывать разный порядок слов в разных языках.

<img src ="http://edunet.kea.su/repo/src/L08_RNN/img/498_FA2019_lecture13-025.png" width="700">

* Механизм внимания не обязательно должен принимать на вход последовательность. 

* Мы можем применять его, в том числе для генерации подписей для картинок. Входом в данном случае будет являться сетка признаков, которая была получена при применении сверточной сети к картинке. 

* Далее по этой сетке мы считаем веса внимания и делаем аналогично первому примеру.

### Image Captioning with RNNs and Attention

Модели, основанные на внимании (attention) намного более продвинутые, нежели обычные нейросети. Они могут концентрироваться на отдельных частях изображения, что позволяет избежать зашумления данных.

Идея состоит в том, что свёрточная сеть теперь будет генерировать не один вектор, описывающий всё изображение, а набор векторов для нескольких участков исходного снимка. В дополнение к работе со словарём на каждом временном шаге модель также производит распределение по точкам на изображении, которые она обрабатывает в данный момент. Это позволяет ей научиться находить наиболее важные участки, на которых необходимо фокусироваться.

После обучения модели можно увидеть, что она как бы переносит своё внимание по изображению для каждого генерируемого слова.

<img src ="http://edunet.kea.su/repo/src/L08_RNN/img/498_FA2019_lecture13-028.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L08_RNN/img/498_FA2019_lecture13-031.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L08_RNN/img/498_FA2019_lecture13-036.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L08_RNN/img/498_FA2019_lecture13-037.png" width="700">



<img src ="http://edunet.kea.su/repo/src/L08_RNN/img/498_FA2019_lecture13-038.png" width="700">

Существуют также понятия мягкого и жёсткого внимания (soft and hard attention). При мягком внимании мы берём взвешенную комбинацию признаков по всему изображению, тогда как в случае жёсткого внимания мы заставляем модель выбирать только один небольшой участок для обработки на каждом временном шаге. При этом жёсткое внимание, строго говоря, не является дифференцируемой функцией. Поэтому для обучения такой модели необходимо использовать более изощрённые приёмы, чем обычное обратное распространение ошибки.

Также нейросети, основанные на внимании, повсеместно используются для ответов на визуальные вопросы (Visual Question Answering). Цель этой задачи — обучить модель отвечать на вопрос по изображению. Например, она должна уметь не только называть сами объекты на фотографии, но и считать их, распознавать цвета и оценивать расположение относительно друг друга. Мы уже рассказывали о подобных архитектурах в статье о том, как такие нейросети могут помочь незрячим людям и о нейро-символическом мышлении.

ссылка https://www.reg.ru/blog/nejroset-opisyvaet-mir-nezryachim-lyudyam/ ссылка https://www.reg.ru/blog/uchim-nejroseti-rassuzhdat-o-tom-chto-oni-vidyat/

<img src ="http://edunet.kea.su/repo/src/L08_RNN/img/498_FA2019_lecture13-039.png" width="700">

### X, Attend, and Y

<img src ="http://edunet.kea.su/repo/src/L08_RNN/img/498_FA2019_lecture13-044.png" width="700">

### Attention Layer

<img src ="http://edunet.kea.su/repo/src/L08_RNN/img/498_FA2019_lecture13-045.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L08_RNN/img/498_FA2019_lecture13-048.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L08_RNN/img/498_FA2019_lecture13-050.png" width="700">

Давайте обобщим наш attention слой. 

* Во-первых, заменим фцию $F_{att}$ на скалярное произведение.

* В предыдущем примере мы использовали только один вектор для поиска совпадений (query вектор), который являлся внутренним состоянием декодера. Давайте вместо этого дадим возможность нашей модели использовать сразу нессколько query-векторов. Благодаря этому мы будем получать несколько распределений весов на выходе (по 1 для каждого query-вектора).

* Обратим внимание, что мы используем входной вектор Х дважды: при вычислении похожести с query-вектором и при вычислении конечного выходного взвешенного решения. Для большей гибкости давайте разделим их и будем отдельно вычислять key-вектора и отдельно value-вектора. key-вектора мы будем использовать для поиска схожести с value-векторами. А они в свою очередь, будут использоваться для вычисления конечного выходного взевешенного значения. 


<img src ="http://edunet.kea.su/repo/src/L08_RNN/img/498_FA2019_lecture13-056.png" width="700">

### Self-Attention Layer

Self-attention, sometimes called intra-attention is an attention mechanism relating different positions
of a single sequence in order to compute a representation of the sequence. Self-attention has been
used successfully in a variety of tasks including reading comprehension, abstractive summarization,
textual entailment and learning task-independent sentence representations

Self-attention, также иногда называемый intra-attention - это механизм , связывающий различные позиции одной последовательности для вычисления представления последовательности. 

Self-attention успешно используется в различных задачах, включая понимание прочитанного (reading comprehension), абстрактное обобщение, текстовое влечение (entailment) и обучение независимым от задачи представлениям предложений.

Возникает вопрос, как это сделать наилучшим образом. 

Допустим, при обработке временных рядов используют некую сглаживающую функцию, например gaussian. 

Однако при обработке предложений очевидно, что неправильно считать рядом стоящие слова как наиболее близкие
друг к другу по контексту.

<img src ="http://edunet.kea.su/repo/src/L08_RNN/img/l08_4-1.png" width="700">

Основная идея заключается в том, что мы рассчитываем и query, и input, и key на основе input-векторов. 

Таким образом мы будем сравнивать каждый наш входной вектор со всеми остальными входными векторами входного множества. 

Для расчета query-векторов мы добавляем матрицу $W_Q$

<img src ="http://edunet.kea.su/repo/src/L08_RNN/img/498_FA2019_lecture13-063.png" width="700">

Свой self attention инвариантен относительно порядка входных последовательностей. 

Таким образом мы получили новый тип слоев нейронной сети, который **не** использует информацию о порядке входной последовательности и оперирует только на множестве входных векторов.

<img src ="http://edunet.kea.su/repo/src/L08_RNN/img/498_FA2019_lecture13-070.png" width="700">

Иногда нам хочется чтобы порядок учитывался, например при переводе и генерации описания картинок (caption). 

Для этого мы добавляем информацию на входы о порядке наших элементов.

<img src ="http://edunet.kea.su/repo/src/L08_RNN/img/498_FA2019_lecture13-072.png" width="700">

### Masked Self-Attention Layer - совместить

<img src ="http://edunet.kea.su/repo/src/L08_RNN/img/498_FA2019_lecture13-073.png" width="700">

#### Multihead Self-Attention Layer

<img src ="http://edunet.kea.su/repo/src/L08_RNN/img/l08_7-1.png" width="700">



Давайте посмотрим на то, к каким словам предложения gave может иметь отношение. В общим случае глагол может иметь связку со многоми частями предложения. Как, например, к подлежащему, так и к причастиям.

В идеале, нам бы хотелось обратить внимние функции (attention) на все эти взаимосвязи.

**Для этого небходимо ввести несколько механизмов внимания, как это показано на картинке**

 <img src ="http://edunet.kea.su/repo/src/L08_RNN/img/l08_6-1.png" width="700"> 

* Чтобы осуществить задуманное, вместо одного набора query, будем использовать несколько независимых наборов. 

* Причем каждый набор будет считаться уникальной матрицей. 

* Аналогично сделаем для keys и values. Количество таких наборов внутри keys, queries, values должно быть **одинаковым**. 

* Обозначим это число как h - head, далее производим аналогичные манипуляции, при этом введем в параллель h таких функций attention
* На последнем шаге мы их соединяем (конкатинируем)

<img src ="http://edunet.kea.su/repo/src/L08_RNN/img/498_FA2019_lecture13-075.png" width="700">

### Three Ways of Processing Sequences

<img src ="http://edunet.kea.su/repo/src/L08_RNN/img/l08_1_1.png" width="700">


Модель attention позволяет обрабатывать последовательности параллельно в отличие от моделей основанных на RNN, это позволяет данную модель в большей мере использовать возможности использования параллельных вычислений на GPU 

RNN:
**Преимущества** 

• Возможность обработки входных данных любой длины

• Размер модели не увеличивается с размером входного сигнала

• Расчет учитывает историческую информацию

• Веса распределяются во времени 

**Недостатки**

• Вычисление происходит медленно

• Трудность доступа к информации, полученной давным-давно

• Не может учитывать какие-либо будущие входные данные для текущего состояния

<img src ="http://edunet.kea.su/repo/src/L08_RNN/img/498_FA2019_lecture13-084.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L08_RNN/img/498_FA2019_lecture13-085.png" width="700">

### The Transformer

Раньше мы передавали информацию из блока attention в RNN для ее последующего применения, однако в 2017 году вышла статья под названием Attention is all you need. В ней говорилось о том, что для обработки последовательностей можно ограничиться только блоком внимания. Данная модель получила название Transformer. 

По сути это базовый блок, который основывается только на self-attention при работе с входными векорами. 

<img src ="http://edunet.kea.su/repo/src/L08_RNN/img/498_FA2019_lecture13-092.png" width="700">

Блок устроен следующим образом: входные вектора мы запускаем в блок self-attention и используем residual connection как обходной путь вокруг self-attention, затем мы их складываем. 

После - применяем слой нормализации, затем - слой feat forward сетей, плюс обходной путь вокруг feat forward. 

Складываем их и выполняем нормализацию. 

Это один блок-трансформер. 

В реальных моделях эти слои стекируют друг с другом и получается большая трансформер-модель. 

<img src ="http://edunet.kea.su/repo/src/L08_RNN/img/498_FA2019_lecture13-094.png" width="700">

Although this achieves parallelization, it is still computationally expensive. The number of operations per layer incurred by RNNs and CNNs is way more unreasonable as compared to the quality of results they offer. The original Transformer paper has put forth a comparison of these parameters for the competent models:

Хотя таким образом (свертками) достигается распараллеливание, однако это вычислительно дорого. Количество операций на слой, выполняемых RNN и CNN, гораздо более необоснованно по сравнению с качеством результатов, которые они предлагают.

В одной из статей про Трансформеры было предложено сравнение этих параметров для компетентных моделей:

<img src ="http://edunet.kea.su/repo/src/L08_RNN/img/L08_trans.png" width="700">

Here, d (or d_model) is the representation dimension or embedding dimension of a word (usually in the range 128–512), n is the sequence length (usually in the range 40–70), k is the kernel size of the convolution and r is the attention window-size for restricted self-attention. From the table, we can infer the following:

* Clearly, the per-layer computational complexity of self-attention is way less than that of others.
* With respect to sequential operations, except RNNs, all other approaches offer parallelization, hence their complexity is O(1).
* The final metric is maximum path length, which superficially means the complexity for attending long-term dependencies or distant words. Since convolutional models use hierarchical representations, their complexity is nlog(n), while self-attention models attend all the words at the same step, hence their complexity is O(1).


The Transformer uses the self-attention mechanism where attention weights are calculated using all the words in the input sequence at once, hence it facilitates parallelization. In addition to that, since the per-layer operations in the Transformer are among words of the same sequence, the complexity does not exceed O(n²d). Hence, the transformer proves to be effective (since it uses attention) and at the same time, a computationally efficient model.

Здесь d (или *d_model*) - размер выборки или embedding вложения слова (обычно в диапазоне 128-512), n-длина последовательности (обычно в диапазоне 40-70), k-размер ядра свертки, а r-размер окна внимания для ограниченного self-attention. Из таблицы мы можем сделать следующие выводы:

* Очевидно, что вычислительная сложность каждого слоя self-attention намного меньше, чем у других.
* Что касается последовательных операций, то, за исключением RNN, все другие подходы предлагают параллельность действий, поэтому их сложность составляет O(1).
* Конечная метрика - максимальная длина пути, которая поверхностно означает сложность для посещения долгосрочных зависимостей или отдаленных слов. Поскольку сверточные модели используют иерархические представления, их сложность равна nlog(n), в то время как модели самосознания посещают все слова на одном и том же шаге, следовательно, их сложность равна O(1).


Трансформер использует механизм само-внимания, где веса внимания вычисляются с использованием всех слов во входной последовательности сразу, следовательно, это облегчает распараллеливание. Кроме того, поскольку операции на каждом слое в трансформерах относятся к словам одной и той же последовательности, сложность не превышает O($n^2d$). Следовательно, трансформер оказывается эффективной (поскольку она использует attention) и в то же время вычислительно качественной моделью.

статья: https://towardsdatascience.com/transformers-explained-65454c0f3fa7

### The Transformer: Transfer Learning

<img src ="http://edunet.kea.su/repo/src/L08_RNN/img/498_FA2019_lecture13-095.png" width="700">

В 2020 практически все успехи машинного обучения были связаны с применением трансформеров. На сегодняшний день - это самая передовая модель для обработки последовательностей. 

Стоит отметить, что модели трансформеров имеют огромное кличество параметров, как следствие - требуются большие вычислительные мощности для их обучения. В связи с этим, обучение данных моделей с нуля, преимущественно доступны только развитым государствам и крупным компаниям. 

Большая часть предприятий может расчитывать только на дообучене этих моделей для своих задач, что все равно показывает хорошие результаты.

### The Transformer: Transfer Learning

<img src ="http://edunet.kea.su/repo/src/L08_RNN/img/498_FA2019_lecture13-100.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L08_RNN/img/498_FA2019_lecture13-104.png" width="700">

### Summary

<img src ="http://edunet.kea.su/repo/src/L08_RNN/img/498_FA2019_lecture13-105.png" width="700">

## Image Captioning



Суть этой задачи заключается в том, чтобы нейросеть составила текстовое описание фотографии. Для этого необходимо сначала классифицировать объекты на изображении, а затем передать результат (одну или несколько меток) в языковую рекуррентную модель, которая сможет составить из них осмысленную фразу. При этом мы действуем точно так же, как в случае с обычной языковой моделью: преобразуем метку изображения в вектор, который обрабатывается декодером.

Чтобы рекуррентная сеть понимала, где именно начинается предложение, во время обучения на её вход подаётся стартовый опознавательный знак ( token $ <start>$ ). Для построения фразы используется заранее подготовленный словарь, например, из английских слов — и он может быть довольно большим.

При переходе на каждое следующее скрытое состояние мы сохраняем как уже сгенерированные слова, так и информацию об изображении. В конце предложения в нейросеть отправляется финальный токен ( $ <end> $). Во время тестирования модель уже самостоятельно определяет, где должно начинаться и заканчиваться описание изображения.

Обычно подобные архитектуры создаются с помощью контролируемого обучения (supervized learning) — это означает, что в обучающих датасетах уже присутствуют как изображения, так и описания для них. Наиболее популярным и самым большим набором данных является Microsoft COCO. Помимо image captioning он также применяется для сегментации, поиска ключевых точек и даже построения трёхмерной модели человека на основе его позы.

<img src ="http://edunet.kea.su/repo/src/L08_RNN/img/498_FA2019_lecture12-071.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L08_RNN/img/498_FA2019_lecture12-079.png" width="700">

Мы добавили в словарь две специальные команды $ <start> $ и $ <end> $ 

Иногда дает очень хорошие описания

<img src ="http://edunet.kea.su/repo/src/L08_RNN/img/498_FA2019_lecture12-080.png" width="1000">

# Пример машинного перевода



Подробнее о том, как выглядят нейросети для машинного перевода и, в частности, Google-переводчик, можно прочитать в статье Google преодолевает барьер между человеческим и машинным переводом. А мы вернёмся к распознаванию изображений и поговорим об image captioning.

ссылка https://www.reg.ru/blog/google-preodolevaet-barer-mezhdu-chelovecheskim-i-mashinnym-perevodom/

# Применение RNN для генерации данных.

<img src ="http://edunet.kea.su/repo/src/L08_RNN/img/498_FA2019_lecture12-012.gif" width="300">

<img src ="http://edunet.kea.su/repo/src/L08_RNN/img/498_FA2019_lecture12-013.png" width="400">

<img src ="http://edunet.kea.su/repo/src/L08_RNN/img/498_FA2019_lecture12-013-1.png" width="400">

<img src ="http://edunet.kea.su/repo/src/L08_RNN/img/498_FA2019_lecture12-014.gif" width="400">

# Чулан

<img src ="http://edunet.kea.su/repo/src/L08_RNN/img/L08_14.png" width="800">

* Rnn обеспечивает большую гибкость в архитектурном проектировании
* Vanilla RNN намного проще, однако работают не очень хорошо
* LSTM или GRU: аддитивное взаимодействие улучшает градиентный поток
* Обратный поток градиентов в RNN делает explode или vanish

explode управляется с помощью градиентного отсечения

vanish - это контролируемые аддитивные взаимодействия (LSTM)

* Лучшие/более простые архитектуры являются актуальной темой текущих исследований
* Необходимо лучшее понимание (как теоретическое, так и эмпирическое)